#  Usage/Benchmark Tests
-------------------


In [1]:
# Deps
using CSV
using BenchmarkTools
using Quack;
using Profile

┌ Info: Precompiling Quack [332d158b-1d50-4650-beb3-f7fe5de64c74]
└ @ Base loading.jl:1260


## Testing on 1.0M Random Points 
...And then removing 150,000 and reindexing the QTree

In [3]:
const QT_TEST_SIZE, RM_TEST_SIZE = 1_000_000, 150_000

## Inserts - Tree Build
ps = Quack.genRandomCoords(0., 1., 0., 1., QT_TEST_SIZE);

sampleQTree = Quack.qtBox(Quack.Coord(.0, .0), 1.)
@time (for p in ps Quack.insertIntoQuadTree!(sampleQTree, p) end)

  2.684615 seconds (26.54 M allocations: 943.413 MiB, 21.77% gc time)


In [3]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")

Total Obj Size: 81.672808 MB


In [4]:
@benchmark Quack.queryRange(sampleQTree, Quack.Box(Quack.Coord(rand() , rand()), rand()/2))

BenchmarkTools.Trial: 
  memory estimate:  14.20 KiB
  allocs estimate:  218
  --------------
  minimum time:     20.943 μs (0.00% GC)
  median time:      12.020 ms (0.00% GC)
  mean time:        22.095 ms (11.63% GC)
  maximum time:     106.790 ms (9.46% GC)
  --------------
  samples:          226
  evals/sample:     1

In [6]:
# Test in kilometers (Rand between 10-20km) 
@benchmark Quack.radialSearch(sampleQTree, Quack.Coord(rand(), rand()), (1 + rand()) * 10)

BenchmarkTools.Trial: 
  memory estimate:  217.16 MiB
  allocs estimate:  2412831
  --------------
  minimum time:     300.111 ms (4.09% GC)
  median time:      339.810 ms (4.48% GC)
  mean time:        344.660 ms (4.34% GC)
  maximum time:     371.899 ms (3.13% GC)
  --------------
  samples:          15
  evals/sample:     1

In [7]:
# Use @time instead of @benchmark so as to not compound the side effect...
@time (for i in 1:RM_TEST_SIZE Quack.removePoint!(sampleQTree, ps[rand(1:QT_TEST_SIZE)]) end)

  1.646411 seconds (11.66 M allocations: 544.880 MiB, 6.11% gc time)


In [8]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")
println("Total Point Count: $(length(Quack.queryRange(sampleQTree, Quack.Box(Quack.Coord(0. , 0.), 1))))")

Total Obj Size: 77.216776 MB
Total Point Count: 860749


In [10]:
# Cleanup/Reinded the QTree - Remove unused Nodes
@time Quack.cleanUpQTree(sampleQTree)

ArgumentError: ArgumentError: reducing over an empty collection is not allowed

In [11]:
println("Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB")
println("Total Point Count: $(length(Quack.queryRange(sampleQTree, Quack.Box(Quack.Coord(0. , 0.), 1))))")

Total Obj Size: 76.428128 MB
Total Point Count: 860749


### Test on "Real" Data

In [2]:
# Read data && generate coords
f = CSV.read("./data/FL_insurance_sample.csv");

samplePoint = Quack.Coord(-82.452606, 27.964157) # 27.964157, -82.452606

ps = [Quack.Coord(x, y) for (x,y) in zip(f[:, :point_longitude], f[:, :point_latitude])];

In [5]:
# Benchmark against raw calculation...
@benchmark filter(p -> p = Quack.haversineDistance(p, samplePoint) < 5, ps)

BenchmarkTools.Trial: 
  memory estimate:  1.12 MiB
  allocs estimate:  3
  --------------
  minimum time:     2.678 ms (0.00% GC)
  median time:      2.870 ms (0.00% GC)
  mean time:        2.969 ms (0.47% GC)
  maximum time:     5.197 ms (22.57% GC)
  --------------
  samples:          1682
  evals/sample:     1

In [3]:
# prep radialSearch implementation...
lngLow, lngHigh = minimum(f[:, :point_longitude]), maximum(f[:, :point_longitude]);
latLow, latHigh = minimum(f[:, :point_latitude]), maximum(f[:, :point_latitude]);

minSideLength = maximum([lngHigh - lngLow, latHigh - latLow])

startCoord = Quack.Coord(lngLow, latLow)

epsilon = 1/1000
sampleQTree = Quack.qtBox(startCoord, minSideLength + epsilon)
@time (for p in ps Quack.insertIntoQuadTree!(sampleQTree, p) end)

  0.072434 seconds (1.06 M allocations: 37.539 MiB, 16.05% gc time)


In [4]:
# Check radialSearch implementation; not yet complete...
@benchmark Quack.radialSearch(sampleQTree, samplePoint, 5.)

BenchmarkTools.Trial: 
  memory estimate:  14.14 KiB
  allocs estimate:  141
  --------------
  minimum time:     17.337 μs (0.00% GC)
  median time:      18.264 μs (0.00% GC)
  mean time:        20.852 μs (4.29% GC)
  maximum time:     3.268 ms (99.31% GC)
  --------------
  samples:          10000
  evals/sample:     1